In [46]:
from models import *
import os
import glob
import argparse
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from idhp_data import *


def _split_output(yt_hat, t, y, y_scaler, x, index):
    q_t0 = y_scaler.inverse_transform(yt_hat[:, 0].copy())
    q_t1 = y_scaler.inverse_transform(yt_hat[:, 1].copy())
    g = yt_hat[:, 2].copy()

    if yt_hat.shape[1] == 4:
        eps = yt_hat[:, 3][0]
    else:
        eps = np.zeros_like(yt_hat[:, 2])

    y = y_scaler.inverse_transform(y.copy())
    var = "average propensity for treated: {} and untreated: {}".format(g[t.squeeze() == 1.].mean(),
                                                                        g[t.squeeze() == 0.].mean())
    print(var)

    return {'q_t0': q_t0, 'q_t1': q_t1, 'g': g, 't': t, 'y': y, 'x': x, 'index': index, 'eps': eps}


In [47]:
def train_and_predict_dragons(t, y_unscaled, x, targeted_regularization=True, output_dir='',
                              knob_loss=dragonnet_loss_binarycross, ratio=1., dragon='', val_split=0.2, batch_size=64):
    verbose = 0
    y_scaler = StandardScaler().fit(y_unscaled)
    y = y_scaler.transform(y_unscaled)
    train_outputs = []
    test_outputs = []

    if dragon == 'tarnet':
        dragonnet = make_tarnet(x.shape[1], 0.01)

    elif dragon == 'dragonnet':
        print("I am here making dragonnet")
        dragonnet = make_dragonnet(x.shape[1], 0.01)

    metrics = [regression_loss, binary_classification_loss, treatment_accuracy, track_epsilon]

    if targeted_regularization:
        loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
    else:
        loss = knob_loss

    # for reporducing the IHDP experimemt

    i = 0
    #tf.random.set_random_seed(i)
    tf.random.set_seed(i)
    np.random.seed(i)
    #print("x.shape::",np.arange(x.shape[0]))
    train_index, test_index = train_test_split(np.arange(x.shape[0]), test_size=val_split, random_state=1)
    #test_index = train_index

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    t_train, t_test = t[train_index], t[test_index]

    yt_train = np.concatenate([y_train, t_train], 1)

    import time;
    start_time = time.time()

    dragonnet.compile(
        optimizer=Adam(lr=1e-3),
        loss=loss, metrics=metrics)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0)

    ]

    dragonnet.fit(x_train, yt_train, callbacks=adam_callbacks,
                  validation_split=val_split,
                  epochs=100,
                  batch_size=batch_size, verbose=verbose)

    sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0)
    ]

    sgd_lr = 1e-5
    momentum = 0.9
    dragonnet.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True), loss=loss,
                      metrics=metrics)
    dragonnet.fit(x_train, yt_train, callbacks=sgd_callbacks,
                  validation_split=val_split,
                  epochs=300,
                  batch_size=batch_size, verbose=verbose)

    elapsed_time = time.time() - start_time
    print("***************************** elapsed_time is: ", elapsed_time)

    yt_hat_test = dragonnet.predict(x_test)
    yt_hat_train = dragonnet.predict(x_train)

    test_outputs += [_split_output(yt_hat_test, t_test, y_test, y_scaler, x_test, test_index)]
    train_outputs += [_split_output(yt_hat_train, t_train, y_train, y_scaler, x_train, train_index)]
    K.clear_session()

    return test_outputs, train_outputs


In [48]:
#turn_knob("/local_home/ag62216/var/dragonnet/dat/ihdp/csv", "dragonnet", "/local_home/ag62216/var/dragonnet/result/ihdp")

In [49]:
data_base_dir = "/local_home/ag62216/var/dragonnet/dat/ihdp/csv"
dragon='dragonnet'
output_dir = os.path.join(data_base_dir, "dragonnet")
output_dir

'/local_home/ag62216/var/dragonnet/dat/ihdp/csv/dragonnet'

In [50]:
#run_ihdp(data_base_dir=data_base_dir, output_dir=output_dir, dragon='dragonnet')
knob_loss=dragonnet_loss_binarycross
ratio=1.

print("the dragon is {}".format(dragon))

simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))

print(simulation_files)

the dragon is dragonnet
['/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_1.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_10.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_11.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_12.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_13.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_14.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_15.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_16.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_17.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_18.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_19.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_2.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_20.csv', '/local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_21.csv', '/local_home/ag62216/var/dragonnet/dat/

In [51]:
#for idx, simulation_file in enumerate(simulation_files):
idx = 0 
simulation_file = simulation_files[0]
print(idx, simulation_file)
simulation_output_dir = os.path.join(output_dir, str(idx))
print("simulation_output_dir:",simulation_output_dir)

os.makedirs(simulation_output_dir, exist_ok=True)

x = load_and_format_covariates_ihdp(simulation_file)
t, y, y_cf, mu_0, mu_1 = load_all_other_crap(simulation_file)
np.savez_compressed(os.path.join(simulation_output_dir, "simulation_outputs.npz"),
                    t=t, y=y, y_cf=y_cf, mu_0=mu_0, mu_1=mu_1)

print("x::",type(x),x.shape)
print(x[0])
print("t::",type(t),t.shape)
print(t[0:10])
print("y::",type(y),y.shape)
print(y[0:10])

0 /local_home/ag62216/var/dragonnet/dat/ihdp/csv/ihdp_npci_1.csv
simulation_output_dir: /local_home/ag62216/var/dragonnet/dat/ihdp/csv/dragonnet/0
x:: <class 'numpy.ndarray'> (747, 25)
[ 1.          0.          1.          0.          0.          0.
  0.          1.          0.          1.          1.          1.
  1.          0.          0.          0.          0.          0.
  0.         -0.52860282 -0.3434545   1.12855393  0.16170253 -0.31660318
  1.29521594]
t:: <class 'numpy.ndarray'> (747, 1)
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
y:: <class 'numpy.ndarray'> (747, 1)
[[5.59991629]
 [6.87585616]
 [2.99627271]
 [1.36620569]
 [1.96353814]
 [4.76209035]
 [6.59404386]
 [2.90823459]
 [2.13134649]
 [2.60232276]]


In [56]:
#for is_targeted_regularization in [True, False]:
is_targeted_regularization = True 
print("Is targeted regularization: {}".format(is_targeted_regularization))

test_outputs, train_output = train_and_predict_dragons(t, y, x,
                                                       targeted_regularization=is_targeted_regularization,
                                                       output_dir=simulation_output_dir,
                                                       knob_loss=knob_loss, ratio=ratio, dragon=dragon,
                                                       val_split=0.2, batch_size=64)

Is targeted regularization: True
I am here making dragonnet
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
***************************** elapsed_time is:  8.488697290420532
average propensity for treated: 0.2091255933046341 and untreated: 0.16319814324378967
average propensity for treated: 0.2397436946630478 and untreated: 0.16083696484565735


In [ ]:
test_outputs, train_output = train_and_predict_dragons(t, y, x,
                                                       targeted_regularization=is_targeted_regularization,
                                                       output_dir=simulation_output_dir,
                                                       knob_loss=knob_loss, ratio=ratio, dragon=dragon,
                                                       val_split=0.2, batch_size=747)

I am here making dragonnet
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
